In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 62.1 MB/s 
     |████████████████████████████████| 182 kB 63.2 MB/s 


In [2]:
from google.colab import drive
import pandas as pd
import re 
import torch
import transformers 
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import AutoModel, AutoTokenizer 

from transformers import pipeline


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_data_path = "/content/drive/MyDrive/Colab Notebooks/webis-clickbait-22/train.jsonl"
df = pd.read_json(train_data_path, lines=True)

train_data = df[['postText','targetParagraphs', 'spoiler', 'postPlatform', 'tags']].astype(str)

#train_data['postText_str'] = [''.join(map(str, l)) for l in train_data['postText']]
#train_data['targetParagraphs_str'] = [''.join(map(str, l)) for l in train_data['targetParagraphs']]
#train_data['spoiler_str'] = [''.join(map(str, l)) for l in train_data['spoiler']]

#train_data[['postText_str', 'targetParagraphs_str', 'spoiler_str']] = train_data[['postText_str', 'targetParagraphs_str', 'spoiler_str']].replace(r'\W', ' ', regex = True)

#train_data = train_data.applymap(str)

#train_data['postText_str'] = train_data['postText_str'].astype(str)
#train_data['targetParagraphs_str'] = train_data['targetParagraphs_str'].astype(str)
#train_data['spoiler_str'] = train_data['spoiler_str'].astype(str)

In [5]:
model_name = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [6]:
#whats the optimal length / stride?

max_length = 500 #maximum feature length (question or context)
doc_stride = 128# authorized overlap between two parts of the context when splitting is needed 

In [10]:
df_qa = train_data[['postText', 'targetParagraphs', 'spoiler', 'postPlatform', 'tags']]
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenized_example = tokenizer(
    train_data["postText"][0],
    train_data["targetParagraphs"][0],
    train_data["spoiler"][0],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

# how to optimize truncuation, doc stride and max length?
question_answer_pipeline = pipeline("question-answering", model_name, tokenizer = tokenizer, max_length=500
                                    , truncation=True, return_overflowing_tokens=True, stride=doc_stride, topk=5) # truncation = "only_second"
df_qa['model_prediction'] = ""
df_qa['correct'] = "False"

# Question 14 is long as hell
for i in range(0, len(df_qa[:10])):
    question = df_qa.loc[i]['postText']
    context = df_qa.loc[i]['targetParagraphs']
    answer = df_qa.loc[i]['spoiler']
    tags = df_qa.loc[i]['tags']

    candidates = question_answer_pipeline(question = question, context = context)
    if(tags != "['passage']"):
      results = "['"+candidates[0]['answer']+"']"
      df_qa.loc[i]['model_prediction'] = results
    else:
      for sentence in context.split('.'):
        if candidates[0]['answer'] in sentence:
          sentence = "['"+sentence+"']"
          sentence = re.sub("(',)", "", sentence);
          sentence = re.sub('(",)', "", sentence);
          sentence = re.sub("(' )", "", sentence);
          df_qa.loc[i]['model_prediction'] = sentence
        #else:
        #  result = "['"+result['answer']+"']"
        #  df_qa.loc[i]['model_prediction'] = "['"+result['answer']+"']"
   
   
    if(df_qa.loc[i]['model_prediction'] == df_qa.loc[i]['spoiler']):
      df_qa.loc[i]['correct'] = 'True'
    else:
      df_qa.loc[i]['correct'] = 'False'

In [12]:
# candidates prints the top 5 candidates. We could implement these into the multi however they are oftentimes identical
candidates

[{'score': 0.3707868158817291,
  'start': 1279,
  'end': 1286,
  'answer': 'Tapatío'},
 {'score': 0.10126049816608429,
  'start': 1279,
  'end': 1286,
  'answer': 'Tapatío'},
 {'score': 0.0806351825594902,
  'start': 3813,
  'end': 3837,
  'answer': 'Sriracha Hot Chili Sauce'},
 {'score': 0.05484845116734505,
  'start': 1279,
  'end': 1286,
  'answer': 'Tapatío'},
 {'score': 0.04256237670779228,
  'start': 1275,
  'end': 1286,
  'answer': '15. Tapatío'}]

In [13]:
df_qa[:10]

,postText,targetParagraphs,spoiler,postPlatform,tags,model_prediction,correct
0,"['Wes Welker Wanted Dinner With Tom Brady, But...",['It’ll be just like old times this weekend fo...,['how about that morning we go throw?'],reddit,['passage'],[I’ll get over there early and we can throw a ...,False
1,['NASA sets date for full recovery of ozone ho...,['2070 is shaping up to be a great year for Mo...,['2070'],Twitter,['phrase'],['2070'],True
2,"[""This is what makes employees happy -- and it...","[""Despite common belief, money isn't the key t...",['intellectual stimulation'],Twitter,['phrase'],['intellectually stimulated'],False
3,['Passion is overrated —\xa07 work habits you ...,"['It’s common wisdom. Near gospel really, and ...",['Purpose connects us to something bigger and ...,Twitter,['multi'],['seven'],False
4,"[""The perfect way to cook rice so that it's pe...","['Boiling rice may seem simple, but there is a...",['in a rice cooker'],Twitter,['phrase'],['one piece of equipment'],False
5,['What happens if your new AirPods get lost or...,"[""One of the biggest surprise announcements at...","[""Apple says that if AirPods are lost or stole...",reddit,['passage'],"[""Apple says that if AirPods are lost or stole...",False
6,['The Reason Why Gabor Kiraly Wears THOSE Trac...,"['June 14th 2016 3.3K Shares', ""They may look ...","['""The more good games I had in them, the more...",reddit,['passage'],"['But, why? 'Why would you favour a pair of pa...",False
7,['You’ll Never Believe What This Family Saw in...,"['""It was cold and very foggy, the temperature...",['rainbow colours in the sky and a halo spanni...,Facebook,['passage'],"['"" 'If I ever stepped outside and saw this in...",False
8,['Should you drink Red Wine?'],"['5/5 say yes.', 'Cheers to your health! All f...",['Red wine is clearly the drink of choice if y...,Twitter,['passage'],['['5/5 say yes'],False
9,['Hot Sauce Taste Test: Find out which we name...,['We tried 29 different kinds of hot sauce to ...,"['Sriracha Hot Chili Sauce', ""Frank's RedHot O...",reddit,['multi'],['Tapatío'],False


# Trash from a tutorial, the actual part is above:

In [ ]:
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
train_data.head()

,postText,targetParagraphs,spoiler,postPlatform,spoilerPositions,tags,postText_str,targetParagraphs_str,spoiler_str
0,"['Wes Welker Wanted Dinner With Tom Brady, But...",['It’ll be just like old times this weekend fo...,['how about that morning we go throw?'],reddit,"[[[3, 151], [3, 186]]]",['passage'],Wes Welker Wanted Dinner With Tom Brady But P...,It ll be just like old times this weekend for ...,how about that morning we go throw
1,['NASA sets date for full recovery of ozone ho...,['2070 is shaping up to be a great year for Mo...,['2070'],Twitter,"[[[0, 0], [0, 4]]]",['phrase'],NASA sets date for full recovery of ozone hole,2070 is shaping up to be a great year for Moth...,2070
2,"[""This is what makes employees happy -- and it...","[""Despite common belief, money isn't the key t...",['intellectual stimulation'],Twitter,"[[[1, 186], [1, 210]]]",['phrase'],This is what makes employees happy and it s...,Despite common belief money isn t the key to ...,intellectual stimulation
3,['Passion is overrated —\xa07 work habits you ...,"['It’s common wisdom. Near gospel really, and ...",['Purpose connects us to something bigger and ...,Twitter,"[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...",['multi'],Passion is overrated 7 work habits you need ...,It s common wisdom Near gospel really and no...,Purpose connects us to something bigger and in...
4,"[""The perfect way to cook rice so that it's pe...","['Boiling rice may seem simple, but there is a...",['in a rice cooker'],Twitter,"[[[5, 60], [5, 76]]]",['phrase'],The perfect way to cook rice so that it s perf...,Boiling rice may seem simple but there is a v...,in a rice cooker


## Preprocessing training data
- Preprocessing by using a Transformer tokenizer
- put it in a format the model expects
- generates other inputs


In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Ensures, that our tokenizer is fast (backed by rust)

In [ ]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenizer(train_data['postText_str'][0], train_data['targetParagraphs_str'][0])

{'input_ids': [101, 14008, 2057, 28143, 2099, 2359, 4596, 2007, 3419, 10184, 2021, 11579, 26171, 2018, 2488, 2801, 102, 2009, 2222, 2022, 2074, 2066, 2214, 2335, 2023, 5353, 2005, 3419, 10184, 1998, 14008, 2057, 28143, 2099, 2057, 28143, 2099, 3936, 5958, 2851, 2006, 1037, 5631, 2557, 2276, 2008, 2002, 11925, 10184, 2138, 2002, 2222, 2022, 1999, 2237, 2005, 4465, 1055, 2208, 2090, 1996, 2047, 2563, 11579, 1998, 5631, 13600, 2012, 12267, 7585, 3346, 2009, 2790, 2066, 1037, 3819, 4495, 2005, 1996, 2048, 2000, 4608, 2039, 2021, 10184, 1055, 6210, 1997, 9105, 2039, 7336, 2521, 2062, 2084, 2074, 1037, 7954, 1999, 2755, 2009, 7336, 2070, 18204, 9105, 2004, 1996, 11579, 9074, 3504, 2000, 2994, 4629, 2076, 2010, 2176, 2208, 13366, 13806, 5867, 8636, 1045, 2718, 2032, 2039, 2000, 2079, 4596, 5095, 2305, 2002, 1055, 2066, 1045, 1049, 2183, 2000, 2022, 3909, 1999, 2013, 5754, 19679, 2101, 2044, 1996, 4174, 5169, 2374, 2208, 2021, 2129, 2055, 2008, 2851, 2057, 2175, 5466, 2057, 28143, 2099, 2056, 

Define max_length and doc_stride -> Read into this or ask jüri

In [ ]:
max_length = 500 #maximum feature length (question or context)
doc_stride = 128# authorized overlap between two parts of the context when splitting is needed 

In [ ]:
len(tokenizer(train_data['postText_str'][0], train_data['targetParagraphs_str'][0], max_length=max_length, truncation="only_second")["input_ids"])


353

In [ ]:
tokenized_example = tokenizer(
    train_data["postText_str"][0],
    train_data["targetParagraphs_str"][0],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] wes welker wanted dinner with tom brady but patriots qb had better idea [SEP] it ll be just like old times this weekend for tom brady and wes welker welker revealed friday morning on a miami radio station that he contacted brady because he ll be in town for sunday s game between the new england patriots and miami dolphins at gillette stadium it seemed like a perfect opportunity for the two to catch up but brady s definition of catching up involves far more than just a meal in fact it involves some literal catching as the patriots quarterback looks to stay sharp during his four game deflategate suspension i hit him up to do dinner saturday night he s like i m going to be flying in from ann arbor later after the michigan colorado football game but how about that morning we go throw welker said on wqam per the boston globe and i m just sitting there i m like i was just thinking about dinner but yeah sure i ll get over there early and we can throw a little bit welker was one of brady

In [ ]:
tokenized_example = tokenizer(
    train_data["postText_str"][0],
    train_data["targetParagraphs_str"][0],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 6), (6, 9), (9, 10), (11, 17), (18, 24), (25, 29), (30, 33), (34, 39), (41, 44), (45, 53), (54, 56), (57, 60), (61, 67), (68, 72), (0, 0), (0, 2), (3, 5), (6, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 41), (42, 45), (46, 49), (50, 55), (56, 59), (60, 63), (64, 66), (66, 69), (69, 70), (72, 74), (74, 77), (77, 78), (79, 87), (88, 94), (95, 102), (103, 105), (106, 107), (108, 113), (114, 119), (120, 127), (128, 132), (133, 135), (136, 145), (146, 151), (152, 159), (160, 162), (163, 165), (166, 168), (169, 171), (172, 176), (177, 180), (181, 187), (188, 189), (190, 194), (195, 202), (203, 206), (207, 210), (211, 218), (219, 227), (228, 231), (232, 237), (238, 246), (247, 249), (250, 254), (254, 258), (259, 266), (268, 270), (271, 277), (278, 282), (283, 284), (285, 292), (293, 304), (305, 308), (309, 312), (313, 316), (317, 319), (320, 325), (326, 328), (330, 333), (334, 339), (340, 341), (342, 352), (353, 355), (357, 365), (366, 368), (370, 378), (379

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], train_data["postText_str"][0][offsets[0]:offsets[1]])


wes Wes


In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
train_data["postText_str"] = [q.lstrip() for q in train_data["postText_str"]]

In [ ]:
def prepare_train_features(df):
  for i in range(0,50):
    tokenized_examples = tokenizer(
      train_data["postText_str" if pad_on_right else "targetParagraphs_str"][i],
      train_data["targetParagraphs_str" if pad_on_right else "postText_str"][i],
      truncation="only_second" if pad_on_right else "only_first",
      max_length=max_length,
      stride=doc_stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
    )

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode